# Load

In [ ]:
!gdown --fuzzy https://docs.google.com/spreadsheets/d/1Tl39_L8V8BwGLFkLTKmxJy2xMPTztAcP/edit?usp=sharing&ouid=100299165652406824721&rtpof=true&sd=true

In [ ]:
!pip install -q openpyxl

In [ ]:
import pandas as pd

file_path = '/content/maximum_flow_data.xlsx'

try:
  node = pd.read_excel(file_path, engine='openpyxl', sheet_name="Node")
  edge = pd.read_excel(file_path, engine='openpyxl', sheet_name="Edge")
  matrix = pd.read_excel(file_path, engine='openpyxl', sheet_name="Matrix")
except FileNotFoundError:
  print(f"Error: File '{file_path}' not found.")
except Exception as e:
  print(f"An error occurred: {e}")

# Nodes

In [ ]:
df_node = node.dropna(subset=['Node','Location'])[['Node','Location']]
df_node[['latitude', 'longitude']] = df_node['Location'].str.split(',', expand=True)
df_node['latitude'] = df_node['latitude'].astype(float)
df_node['longitude'] = df_node['longitude'].astype(float)
df_node.rename(columns={'Node': 'name'}, inplace=True)
df_node['name'] = df_node['name'].str.strip()
df_node = df_node.drop(columns=['Location'])
df_node = df_node.sort_values(by='name')
df_node.reset_index(drop=True, inplace=True)
df_node['id'] = range(0, len(df_node))
df_node.to_csv('nodes.csv', encoding='utf-8', index=False)

# Edges

In [ ]:
df_edge = edge.dropna()
df_edge = df_edge.sort_values(by=['Path'])
df_edge = df_edge.drop(columns=['Path'])
df_edge.rename(columns={'From': 'start', 'To': 'end', 'Weight': 'weight'}, inplace=True)
df_edge['start'] = df_edge['start'].str.strip()
df_edge['end'] = df_edge['end'].str.strip()
df_edge['weight'] = df_edge['weight'].astype(int)

node_id_map = dict(zip(df_node['name'], df_node['id']))

df_edge['start'] = df_edge['start'].map(node_id_map)
df_edge['end'] = df_edge['end'].map(node_id_map)

df_edge.to_csv('edges.csv', encoding='utf-8', index=False)

# Adjacent Matrix

In [ ]:
import numpy as np
import pickle

num_nodes = len(df_node)
adj_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

for _, row in df_edge.iterrows():
    start_node = row['start']
    end_node = row['end']
    weight = row['weight']
    adj_matrix[start_node, end_node] = weight

df_node_dict = df_node.set_index('id').to_dict('index')

# Export

In [ ]:
!gdown --fuzzy https://drive.google.com/file/d/18u8HO4oBJB1Bv5YYMvQ9DUzHbodiPBW4/view?usp=sharing

In [ ]:
import pickle

# data_to_save = {
#     'df_node': df_node,
#     'df_edge': df_edge,
#     'adj_matrix': adj_matrix,
#     'df_node_dict': df_node_dict
# }

# with open('maximum_flow_data.pickle', 'wb') as f:
#     pickle.dump(data_to_save, f)

with open('maximum_flow_data.pickle', 'rb') as f:
    loaded_data = pickle.load(f)

df_node = loaded_data['df_node']
df_edge = loaded_data['df_edge']
adj_matrix = loaded_data['adj_matrix']
df_node_dict = loaded_data['df_node_dict']